In [150]:
# !pip install jupyter
# !pip install scikit-learn
# !pip install matplotlib

In [79]:
import pandas as pd

data = pd.read_csv("./train.csv")
original = pd.read_csv("./machine failure.csv")

data.shape

(136429, 14)

In [152]:
data.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [153]:
data.isnull().sum()

id                         0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

Step 1: Data Cleaning is Done.
Step 2: Now will do Data Normalization and view the distribution of each column values in plots

Now all the data is in Numbers, lets normalize it !!!

In [80]:
# from sklearn.preprocessing import StandardScaler # Converts data to have mean = 0 , Standard Deviation = 1. Assumes data = Gaussian bell curve
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample

def data_cleaning(data: pd.DataFrame):
    data[['Product ID Type', 'Product ID Num']] = data["Product ID"].apply(lambda x: pd.Series([x[0], x[1:]]))
    data.drop("Product ID", inplace=True, axis=1)

    if data["Type"].equals(data["Product ID Type"]):
        data.drop("Product ID Type", inplace=True, axis=1)
    data["Type"].replace(("L", "M", "H"), (0, 1, 2), inplace = True)
    print("CLEANED DATA TABLE", data.head())
    return data


# def normalize(data: pd.DataFrame):
#       data_normalized = pd.DataFrame(MinMaxScaler().fit_transform(data), columns=data.columns)
#       return data_normalized


BALANCE DATA NOW

In [81]:
data = data_cleaning(data)
data.drop("id", inplace=True, axis=1)

original = data_cleaning(original)
original.drop('UDI', axis=1, inplace=True)

data = pd.concat([data, original], axis = 0).reset_index(drop=True)


# data["Machine failure"].unique()
# data.insert(13, "output", data["Machine failure"])
# data.drop("Machine failure", inplace=True, axis=1)



# data_normalized = normalize(data)
# print("NORMALIZED DATA TABLE", data_normalized.head())
# print(data_normalized["output"].plot(kind="kde"))
# print(data_normalized["output"].value_counts())


CLEANED DATA TABLE    id  Type  Air temperature [K]  Process temperature [K]  \
0   0     0                300.6                    309.6   
1   1     1                302.6                    312.1   
2   2     0                299.3                    308.5   
3   3     0                301.0                    310.9   
4   4     1                298.0                    309.0   

   Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  Machine failure  TWF  \
0                    1596         36.1              140                0    0   
1                    1759         29.1              200                0    0   
2                    1805         26.5               25                0    0   
3                    1524         44.3              197                0    0   
4                    1641         35.4               34                0    0   

   HDF  PWF  OSF  RNF Product ID Num  
0    0    0    0    0          50096  
1    0    0    0    0          20343  
2    0    

In [82]:

# Divide data into major and minor samples
def resampling(data_normalized: pd.DataFrame):
      print(data_normalized["Machine failure"].value_counts())

      data_majority = pd.DataFrame(data_normalized[data_normalized["Machine failure"]==0])
      data_minority = pd.DataFrame(data_normalized[data_normalized["Machine failure"]==1])

      data_minority_upsampled = resample(data_minority, replace=True,n_samples=134281, random_state=42)
      print(data_minority_upsampled.shape)

      data_upsampled = pd.concat([data_majority, data_minority_upsampled])

      print(data_upsampled["Machine failure"].value_counts())
      return data_upsampled
data_upsampled = resampling(data)

Machine failure
0    143942
1      2487
Name: count, dtype: int64
(134281, 13)
Machine failure
0    143942
1    134281
Name: count, dtype: int64


In [83]:
#split data in to train and test#
from sklearn.model_selection import train_test_split

y_data = data_upsampled["Machine failure"]
print(y_data.shape)


x_data = data_upsampled.drop(["Machine failure"], axis=1) # Use inplace = True ; for deleting the Column in original Dataframe
print(x_data.shape)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

(278223,)
(278223, 12)


In [84]:
# from sklearn.naive_bayes import GaussianNB
# algo = GaussianNB()

# from sklearn.tree import DecisionTreeClassifier
# algo = DecisionTreeClassifier()

# from sklearn.ensemble import RandomForestClassifier
# algo = RandomForestClassifier()

# from sklearn.svm import SVC
# algo = SVC(gamma="auto")

# from sklearn.neighbors import KNeighborsClassifier
# algo = KNeighborsClassifier()

from catboost import Pool, CatBoostClassifier
algo = CatBoostClassifier()

In [85]:
from sklearn.metrics import accuracy_score

# feature_names = ['Product ID','Type',
#                  'Air temperature [K]','Process temperature [K]',
#                  'Rotational speed [rpm]','Torque [Nm]',
#                  'Tool wear [min]','TWF','HDF',
#                  'PWF','OSF','RNF']

# train_pool = Pool(x_train.to_numpy(), y_train.to_numpy(), feature_names=feature_names, 
#                   cat_features=['Product ID', 'Type'])

# model = algo.fit(train_pool, eval_set=train_pool)

model = algo.fit(x_train, y_train)

y_pred = model.predict(x_test)

print(x_test.shape[0], (y_test!=y_pred).sum())
print(accuracy_score(y_test, y_pred))
# gaussian 88.5
# decisiontree = 99.79
# rfc = 99.98
# knn = 99.49

Learning rate set to 0.103588
0:	learn: 0.5206298	total: 15.2ms	remaining: 15.2s
1:	learn: 0.4184670	total: 28.2ms	remaining: 14.1s
2:	learn: 0.3731373	total: 40.9ms	remaining: 13.6s
3:	learn: 0.3285522	total: 54.2ms	remaining: 13.5s
4:	learn: 0.3052070	total: 66.8ms	remaining: 13.3s
5:	learn: 0.2805069	total: 79.4ms	remaining: 13.2s
6:	learn: 0.2662098	total: 92.1ms	remaining: 13.1s
7:	learn: 0.2541335	total: 103ms	remaining: 12.8s
8:	learn: 0.2436903	total: 114ms	remaining: 12.5s
9:	learn: 0.2355430	total: 124ms	remaining: 12.3s
10:	learn: 0.2295536	total: 137ms	remaining: 12.3s
11:	learn: 0.2246915	total: 150ms	remaining: 12.4s
12:	learn: 0.2199325	total: 161ms	remaining: 12.3s
13:	learn: 0.2164527	total: 175ms	remaining: 12.3s
14:	learn: 0.2135893	total: 187ms	remaining: 12.3s
15:	learn: 0.2106342	total: 198ms	remaining: 12.2s
16:	learn: 0.2081124	total: 211ms	remaining: 12.2s
17:	learn: 0.2057091	total: 225ms	remaining: 12.3s
18:	learn: 0.2038263	total: 239ms	remaining: 12.3s
19:	

In [86]:
test_data=pd.read_csv("./test.csv")

test_data = data_cleaning(test_data)
test_data.drop("id", inplace=True, axis=1)

y_pred = model.predict(test_data)



CLEANED DATA TABLE        id  Type  Air temperature [K]  Process temperature [K]  \
0  136429     0                302.3                    311.5   
1  136430     0                301.7                    311.0   
2  136431     0                301.3                    310.4   
3  136432     1                300.1                    309.6   
4  136433     1                303.4                    312.3   

   Rotational speed [rpm]  Torque [Nm]  Tool wear [min]  TWF  HDF  PWF  OSF  \
0                    1499         38.0               60    0    0    0    0   
1                    1713         28.8               17    0    0    0    0   
2                    1525         37.7               96    0    0    0    0   
3                    1479         47.6                5    0    0    0    0   
4                    1515         41.3              114    0    0    0    0   

   RNF Product ID Num  
0    0          50896  
1    0          53866  
2    0          50498  
3    0          212

In [161]:
y_pred = pd.DataFrame(y_pred)
print(y_pred.shape)
print(y_pred[0].unique())
print(y_pred[0].value_counts())
print(y_pred.head())

(90954, 1)
[0. 1.]
0
0.0    89850
1.0     1104
Name: count, dtype: int64
     0
0  0.0
1  0.0
2  0.0
3  0.0
4  0.0


In [87]:
probability  = model.predict_proba(test_data)[:,1]

test_data_1=pd.read_csv("./test.csv")

final_op = pd.DataFrame({"id": test_data_1["id"], "Machine failure": probability})
final_op

,id,Machine failure
0,136429,0.009059
1,136430,0.012913
2,136431,0.003061
3,136432,0.009113
4,136433,0.033530
...,...,...
90949,227378,0.007237
90950,227379,0.011155
90951,227380,0.001766
90952,227381,0.001379


In [88]:
final_op.to_csv("./submission1234.csv", index=False)
print("Submittion file succesfully saved")

Submittion file succesfully saved
